In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define paths to dataset directories
train_dir = "C:\\Users\\Admin\\Machine Learning\\health ml\\chest_xray\\train"
test_dir = "C:\\Users\\Admin\\Machine Learning\\health ml\\chest_xray\\test"
val_dir = "C:\\Users\\Admin\\Machine Learning\\health ml\\chest_xray\\val"

# Data Augmentation and Preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode="nearest"
)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Increased image size
    batch_size=32,
    class_mode='binary'
)

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Load Pretrained Model (Transfer Learning with MobileNetV2)
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers to retain pretrained features
base_model.trainable = False  

# Custom Layers for Classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)  # Prevent overfitting
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# Corrected output layer connection
output_layer = Dense(1, activation='sigmoid')(x)  # Ensure it connects to `x`

# Define the new model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile Model with Adam Optimizer and Learning Rate Scheduler
initial_learning_rate = 1e-4
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=1000, decay_rate=0.9, staircase=True
)

model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=lr_schedule),
    metrics=['accuracy']
)

# Train the Model
history = model.fit(
    train_generator,
    epochs=15,  # Increased epochs
    validation_data=val_generator
)

# Save Model
model.save("pneumonia_detector_improved.h5")

print("Model training complete and saved!")


In [9]:
from tensorflow.keras.preprocessing import image

def predict_pneumonia(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array)
    
    if prediction[0][0] > 0.5:
        print("Pneumonia Detected!")
    else:
        print("Normal Chest X-Ray")

# Example Usage
predict_pneumonia("C:\\Users\Admin\\Machine Learning\\health ml\\person3_virus_16.jpeg")
predict_pneumonia("C:\\Users\Admin\\Machine Learning\\health ml\\img1.jpeg")
predict_pneumonia("C:\\Users\Admin\\Machine Learning\\health ml\\person48_virus_100.jpeg")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Pneumonia Detected!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


<>:16: SyntaxWarning: invalid escape sequence '\A'
<>:17: SyntaxWarning: invalid escape sequence '\A'
<>:18: SyntaxWarning: invalid escape sequence '\A'
<>:16: SyntaxWarning: invalid escape sequence '\A'
<>:17: SyntaxWarning: invalid escape sequence '\A'
<>:18: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Admin\AppData\Local\Temp\ipykernel_6856\1925861783.py:16: SyntaxWarning: invalid escape sequence '\A'
  predict_pneumonia("C:\\Users\Admin\\Machine Learning\\health ml\\person3_virus_16.jpeg")
C:\Users\Admin\AppData\Local\Temp\ipykernel_6856\1925861783.py:17: SyntaxWarning: invalid escape sequence '\A'
  predict_pneumonia("C:\\Users\Admin\\Machine Learning\\health ml\\img1.jpeg")
C:\Users\Admin\AppData\Local\Temp\ipykernel_6856\1925861783.py:18: SyntaxWarning: invalid escape sequence '\A'
  predict_pneumonia("C:\\Users\Admin\\Machine Learning\\health ml\\person48_virus_100.jpeg")


Normal Chest X-Ray
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Pneumonia Detected!
